# Transformaciones Tidy para los archivos fuente

En este notebook mostraremos los pasos necesarios para llevar cada archivo al formato Tidy y luego unirlos bajo una lógica que siga siendo Tidy. Como primer paso, y para trabajar de forma más compacta, vamos a crear un comprensión de diccionario con DataFrames en vez de leer los archivos uno por uno, esta técnica puede facilitar la lectura del código y puede aumentar nuestra productividad.

In [ ]:
import os
import pandas as pd
import csv

In [ ]:
df = {i: pd.read_excel("data_source/chart{}.xlsx".format(i)) for i in range(1,9)}
df

## Transformaciones generales para todos los archivos

Al observar el primer archivo en el análisis exploratorio anterior, nos damos cuenta de que posee tres columnas de variables (Región, Año del Censo y Acceso a TIC) y una columna con la observación (Valor Porcentaje). Para estandarizar llevaremos a cabo las siguientes operaciones:
* Poner los nombres de región con mayúscula en la inicial y minúscula el resto del texto, usando el método title().
* Añadiremos una columna que indique el origen de los datos (INEI o ENE).
* Cambiar la columna de Censo para que utilice sólo el año como un número entero (menor espacio de almacenamiento en la base de datos), este año será usado en INEI o ENE.
* Consideraremos una columna que indique la variable a medir, en este caso Acceso a TIC, y su valor (Accede, No accede), dejando la observación como "porcentaje".

Estos serán los lineamientos generales para todos los archivos, en las tablas de ENE, el año siempre es 2017, justificamos esta decisión con un poco de información adicional del conjunto de datos:

<img src="img/data_info.png" style="border:2px solid black">

### 1. Transformaciones para chart1.xlsx

In [ ]:
# Seleccionamos el DataFrame 1 en el diccionario, y utilizamos el método title() para modificar las regiones:
df[1]["region"] = df[1]["region"].str.title() 

# Creamos una columna para el origen de los datos con un valor único:
df[1]["data_origin"] = "INEI"

# Cambiamos el nombre de la columna "censo" por "year", luego conservamos sólo los últimos cuatro dígitos del 
# texto (para extraer el año) y finalmente cambiamos el tipo de la columna para que sea numérica:
df[1] = df[1].rename(columns={"censo":"year"})
df[1]["year"] = df[1]["year"].str[-4:] # [-4:] indica que se toma desde el cuarto carácter contando desde el final hacia adelante.
df[1]["year"] = df[1]["year"].astype(int) # Este paso se puede concatenar con el anterior usando .str[-4:].astype(int)

# Creamos una columna para la variable "acceso_tic" y su respuesta, en esta etapa seguiremos una estrategia distinta
# a cambiar el nombre, sólo crearemos una nueva columna copiando el contenido, y más adelante la descartaremos.
df[1]["variable"] = "Acceso a TIC"
df[1]["response"] = df[1]["acceso_tic"]

# Cambiamos el nombre de la columna "valor_porcentaje" y nos aseguramos de que esté como tipo float.
df[1] = df[1].rename(columns={"valor_porcentaje":"percentage"})
# df[1]["percentage"] = df[1]["percentage"].astype(float) 
#La línea no es realmente necesaria, puedes verificar al final que ya está en tipo float, pero la dejo de ejemplo.

# Seleccionamos sólo las columnas que queremos mantener
df[1] = df[1][["region", "data_origin", "year", "variable", "response", "percentage"]]

df[1].head()

La limpieza de los archivos 2, 3 y 4 será similar en estructura, pasaremos a limpiar el archivo 5 y a diseñar una abstracción posteriormente.

### 2. Transformaciones para chart5.xlsx

In [ ]:
df[5]["region"] = df[5]["region"].str.title()

df[5]["data_origin"] = "ENE"

df[5]["year"] = 2017 # En este caso no es necesario cambiar el tipo a int, la creamos de esa forma.

df[5]["variable"] = "Empleó equipos informáticos"
df[5]["response"] = df[5]["empleo_equipos_informaticos"]

df[5] = df[5].rename(columns={"valor_porcentaje": "percentage"})

df[5] = df[5][["region", "data_origin", "year", "variable", "response", "percentage"]]

df[5].head()

### 3. Abstracción

Ahora que tenemos una idea general de los pasos para transformar ambas clases de archivos, podemos establecer las diferencias:
* El origen de datos será distinto para ambos tipos de archivo.
* Requieren líneas distintas para obtener el año.
* El bloque de variable/respuesta será distinto para cada archivo.

Para diseñar una transformación más abstracta, podemos hacer uso de diccionarios, funciones y estructuras de control que nos ayuden.

In [ ]:
# Cargamos los archivos como al principio
df = {i: pd.read_excel("data_source/chart{}.xlsx".format(i)) for i in range(1,9)}

# De esta forma, cuando escriba variable_dict[1] entregará el valor "Acceso a TIC", por ejemplo.
variable_dict = { 
    1: "Acceso a TIC",
    2: "Acceso a Internet",
    3: "Acceso a TV Cable",
    4: "Tipo de Teléfono",
    5: "Empleó Equipos Informáticos",
    6: "Usó Internet",
    7: "Usó Internet para buscar Productos y Servicios",
    8: "Tuvo Problemas de Electricidad"
}

# Esta función entrega la posición de la columna que tiene el valor de response que necesitamos.
def get_response(i): 
    if i in [1,2,3,4]: # Si el archivo es de este tipo, el valor está en la tercera columna, o sea 2.
        return 2
    elif i in [5,6,7,8]: # Al saber que no hay más opciones para i, pudo ser "else: return 1", pero queda de ejemplo.
        return 1
# Hay otra forma incluso más corta de hacer esto, con una sola variable, pero dentro del ciclo que crearemos:
# response_col = 2 if i in [1,2,3,4] else 1

# Transformamos:
for i in range(1,9):
    df[i]["region"] = df[i]["region"].str.title()
    
    df[i]["data_origin"] = "INEI" if i in [1,2,3,4] else "ENE"
    
    if i in [1,2,3,4]:
        df[i] = df[i].rename(columns={"censo":"year"})
        df[i]["year"] = df[i]["year"].str[-4:] 
        df[i]["year"] = df[i]["year"].astype(int)
    else:
        df[i]["year"] = 2017
        
    df[i]["variable"] = variable_dict[i]
    df[i]["response"] = df[i].iloc[:, get_response(i)]
    
    df[i] = df[i].rename(columns={"valor_porcentaje": "percentage"})
    
    df[i] = df[i][["region", "data_origin", "year", "variable", "response", "percentage"]]
    
# Y finalmente concatenamos todas las DataFrames, para eso creamos una lista con ellas, sacándolas del diccionario:
df_list = [df[i] for i in range(1,9)]
df = pd.concat(df_list, ignore_index=True)

df.head()

Nuestro conjunto de datos ya se encuentra en formato Tidy. Adicionalmente podemos analizar las columnas generadas para asegurarnos de que el archivo final está bien.

### 4. Testing

In [ ]:
# Deberíamos tener 25 regiones:
print("Regiones:", len(df["region"].unique()), "\n")

# Dos valores únicos para "data_origin" y "year"
print(df["data_origin"].value_counts(dropna=False), "\n")
print(df["year"].value_counts(dropna=False), "\n")

# 8 valores distintos para "variable"
print(df["variable"].value_counts(dropna=False), "\n")

# Valores distintos para "response", el resultado nos muestra que hay valores repetidos pero está bien.
# En el pipeline de Bamboo sería recomendable convertir esta respuesta binaria en 0 y 1.
print(df["response"].value_counts(dropna=False), "\n")

# Revisamos si algún porcentaje se escapa del intervalo [0,1]:
print(df[(df["percentage"]<=0) | (df["percentage"]>=1)], "\n")

# Y por último chequear que no hay valores nulos en ninguna columna:
print(df.isnull().any())

Para guardar nuestro progreso hasta este punto, crearemos un archivo `.csv` que tenga todos los datos en formato tidy. En el pipeline real no realizaremos este paso, por lo tanto el archivo será almacenado en la carpeta `data_temp`.

In [ ]:
# Indicamos index=False para que los índices del DataFrame no sean guardados en el archivo.
# quoting indica qué comillas (quotes) tendrán ciertos tipos de columna, importamos este estilo desde 
# la librería "csv" para solamente poner comillas en los valores que no sean numéricos.
df.to_csv("data_temp/tidy_file.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

Con esto concluimos nuestras transformaciones, en la siguiente sección estudiaremos el modelo relacional.